In [2]:
import pandas as pd

In [31]:
df = pd.read_csv("/home/alena/repos/scimagojr/scimagojr_full_2020-03-31.csv", sep=";")
df.drop(['Sourceid', 'Issn', 'SJR',
 'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
 'Total Cites (3years)', 'Citable Docs. (3years)', 'Cites / Doc. (2years)',
 'Ref. / Doc.', 'Country', 'Publisher', 'Coverage', 'Categories'], axis=1, inplace=True)

In [32]:
print(df.columns.values)
df.sample(10)

['Unnamed: 0' 'Rank' 'Title' 'Type' 'Area']


,Unnamed: 0,Rank,Title,Type,Area
8054,255,256,Biotechnologia,journal,Biotechnology
48944,362,363,"Periodica Polytechnica, Mechanical Engineering",journal,Mechanical Engineering
21027,637,638,Atlantic World,book series,Cultural Studies
55509,197,198,Journal of Neuro-Ophthalmology,journal,Neurology (clinical)
15424,1703,1704,RealWSN 2015 - Proceedings of the 6th ACM Work...,conference and proceedings,Computer Networks and Communications
53077,84,85,Journal of Iron and Steel Research,journal,Metals and Alloys
66046,30,31,IISE Transactions on Healthcare Systems Engine...,journal,Safety Research
31110,148,149,Waste Forum,journal,Environmental Engineering
56452,41,42,Journal of Reproductive Immunology,journal,Obstetrics and Gynecology
37525,81,82,South Asia: Journal of South Asia Studies,journal,History


In [33]:
df_cs = df.loc[df['Area'].str.contains('computer science', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]

KeyError: 'SJR Quartile'

In [25]:
s1 = pd.merge(df_cs, df_ling, how='inner', on=['Title'])

In [28]:
s1

,Unnamed: 0_x,Rank_x,Sourceid_x,Title,Type_x,Issn_x,SJR_x,SJR Quartile_x,H index_x,Total Docs. (2018)_x,...,Total Refs._y,Total Cites (3years)_y,Citable Docs. (3years)_y,Cites / Doc. (2years)_y,Ref. / Doc._y,Country_y,Publisher_y,Coverage_y,Categories_y,Area_y
0,53,54,144747,Computer Assisted Language Learning,journal,09588221,"1,458",Q1,39,59,...,3016,487,139,"3,29","51,12",United Kingdom,Taylor & Francis,"1990-1995, 2005-ongoing",Computer Science Applications (Q1); Language a...,Language and Linguistics
1,53,54,144747,Computer Assisted Language Learning,journal,09588221,"1,458",Q1,39,59,...,3016,487,139,"3,29","51,12",United Kingdom,Taylor & Francis,"1990-1995, 2005-ongoing",Computer Science Applications (Q1); Language a...,Linguistics and Language
2,68,69,27188,Language Learning and Technology,journal,10943501,"1,338",Q1,62,30,...,1576,352,104,"3,50","52,53",United States,University of Hawaii Press,"1997-1998, 2000-ongoing",Computer Science Applications (Q1); Education ...,Language and Linguistics
3,68,69,27188,Language Learning and Technology,journal,10943501,"1,338",Q1,62,30,...,1576,352,104,"3,50","52,53",United States,University of Hawaii Press,"1997-1998, 2000-ongoing",Computer Science Applications (Q1); Education ...,Linguistics and Language
4,80,81,5800207673,ReCALL,journal,09583440,"1,131",Q1,42,37,...,1348,116,43,"2,09","36,43",United Kingdom,Cambridge University Press,1989-ongoing,Computer Science Applications (Q1); Education ...,Language and Linguistics
5,80,81,5800207673,ReCALL,journal,09583440,"1,131",Q1,42,37,...,1348,116,43,"2,09","36,43",United Kingdom,Cambridge University Press,1989-ongoing,Computer Science Applications (Q1); Education ...,Linguistics and Language
6,210,211,21100865101,Synthesis Lectures on Human Language Technologies,book series,19474040,"0,609",Q2,16,3,...,913,64,12,"3,38","304,33",United States,Morgan and Claypool Publishers,2009-ongoing,Computer Networks and Communications (Q1); Lin...,Linguistics and Language
7,263,264,23671,Speech Communication,journal,01676393,"0,513",Q2,93,110,...,5372,664,254,"2,23","48,84",Netherlands,Elsevier BV,1982-ongoing,Language and Linguistics (Q1); Linguistics and...,Language and Linguistics
8,263,264,23671,Speech Communication,journal,01676393,"0,513",Q2,93,110,...,5372,664,254,"2,23","48,84",Netherlands,Elsevier BV,1982-ongoing,Language and Linguistics (Q1); Linguistics and...,Linguistics and Language
9,296,297,26801,Computational Linguistics,journal,"08912017, 15309312","0,455",Q2,85,30,...,1678,258,73,"2,90","55,93",United States,MIT Press,"1985, 1996-ongoing",Language and Linguistics (Q1); Linguistics and...,Language and Linguistics
